# Tutorial 1: fairkit-learn

In this notebook, you will learn how to use fairkit-learn to train and evaluate machine learning models. This tutorial will use the following models, bias mitigation algorithms, and metrics provided by scikit-learn and AI Fairness 360.

## Models

Because there are a variety of models provided by scikit-learn and AI Fairness 360, we will only use a subset for this assignment. The models you will be evaluating are as follows:

* **Logistic Regression**: a Machine Learning algorithm which is used for the classification problems, it is a predictive analysis algorithm and based on the concept of probability. [More info here.](https://machinelearningmastery.com/logistic-regression-for-machine-learning/) [Scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
* **K Nearest Neighbor Classifier**: a model that classifies data points based on the points that are most similar to it. It uses test data to make an “educated guess” on what an unclassified point should be classified as. [More info here.](https://towardsdatascience.com/machine-learning-basics-with-the-k-nearest-neighbors-algorithm-6a6e71d01761) [Scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
* **Random Forest**: an ensemble machine learning algorithm that is used for classification and regression problems. Random forest applies the technique of bagging (bootstrap aggregating) to decision tree learners. [More info here.](https://towardsdatascience.com/understanding-random-forest-58381e0602d2) [Scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
* **Support Vector Classifier**:  a discriminative classifier formally defined by a separating hyperplane. In other words, given labeled training data (supervised learning), the algorithm outputs an optimal hyperplane which categorizes new examples. In two dimentional space this hyperplane is a line dividing a plane in two parts where in each class lay in either side. [More info here.](https://medium.com/machine-learning-101/chapter-2-svm-support-vector-machine-theory-f0812effc72) [Scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
* **Adversarial Debiasing**: learns a classifier to maximize prediction accuracy and simultaneously reduce an adversary's ability to determine the protected attribute from the predictions. [Documentation.](https://aif360.readthedocs.io/en/latest/modules/inprocessing.html#adversarial-debiasing)

The Adversarial Debiasing model is only available for use when using AI Fairness 360 or fairkit-learn.


## Bias Mitigating Algorithms

When using AI Fairness 360 and fairkit-learn, you will have access to the following bias mitigating pre- and post- processing algorithms:

* **Pre-processing algorithms**
    - *Disparate Impact Remover*: a preprocessing technique that edits feature Values increase group fairness while preserving rank-ordering within groups
    - *Reweighing*: a preprocessing technique that Weights the examples in each (group, label) combination differently to ensure fairness before classification
    
    
* **Post-processing algorithms**
    - *Calibrated Equalized Odds*: a post-processing technique that optimizes over calibrated classifier score outputs to find probabilities with which to change output labels with an equalized odds objective
    - *Reject Option Classification*: a postprocessing technique that gives favorable outcomes to unpriviliged groups and unfavorable outcomes to priviliged groups in a confidence band around the decision boundary with the highest uncertainty 


## Model Evaluation Metrics

To evaluate your trained models, you will be using one or more of the following metrics:

* **Performance metrics**:
    - *Accuracy Score* (UnifiedMetricLibrary.accuracy_score) When evaluating a model with this metric, the goal is to *maximize* the value.
    
    
* **Fairness Metrics**:
    - *Equal Opportunity Difference* (UnifiedMetricLibrary.equal_opportunity_difference) also known as "true positive rate difference". When evaluating a model with this metric, the goal is to *minimize* the value.
    - *Average Odds Difference* (UnifiedMetricLibrary.average_odds_difference) When evaluating a model with this metric, the goal is to *minimize* the value.
    - *Statistical Parity Difference* (UnifiedMetricLibrary.mean_difference) also known as "mean difference". When evaluating a model with this metric, the goal is to *minimize* the value.
    - *Disparate Impact* (UnifiedMetricLibrary.disparate_impact)  When evaluating a model with this metric, the goal is to *maximize* the value.

# Step 1: Import required packages

In [1]:
# Load all necessary packages
import numpy as np
import sklearn as skl
import six
import tensorflow as tf

# datasets
from aif360.datasets import CompasDataset

# metrics
from fklearn.metric_library import UnifiedMetricLibrary, classifier_quality_score

# models
from fklearn.scikit_learn_wrapper import LogisticRegression, KNeighborsClassifier, RandomForestClassifier, SVC
from aif360.algorithms.inprocessing import AdversarialDebiasing

# pre/post-processing algorithms
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, RejectOptionClassification

# model search
from fklearn.fair_selection_aif import ModelSearch, DEFAULT_ADB_PARAMS

# Step 2: Load the dataset

The first thing to do is load the dataset. Below we use the COMPAS recidivism dataset, as provided by AI Fairness 360.

In [2]:
dataset = CompasDataset()

# Step 3: Specify protected attributes

To use the grid search functionality provided by fairkit-learn, we need to specify the privileged and unprivileged (protected) attributes. For the COMPAS dataset, the protected attributes are race and sex.

Below we provide code that stores the protected attributes (*race* is 0 for "Not Caucasian", *sex* is 0 for "Male").

In [3]:
unprivileged = [{'race': 0, 'sex': 0}]
privileged = [{'race': 1, 'sex': 1}]

# Step 4: Specify parameters for model search

Now we need to specify the various parameters required for the grid search provided by fairkit-learn. Each search parameter is a dictionary of options to include in the search. For each search parameter, you can input one or multiple options to consider. 

Below we provide code that sets parameters for a simple grid search across different hyper-parameter values for the Logistic Regression and Random Forest Classifier models, with and without the specified pre-/post-processing algorithms. We specify all performance and fairness metrics for the search -- given the way the classifier quality score is calculated, this cannot be added to the grid search and will be calculated later.

In [4]:
models = {'LogisticRegression': LogisticRegression, 'RandomForestClassifier': RandomForestClassifier }

metrics = {'UnifiedMetricLibrary': [UnifiedMetricLibrary,
                                    'accuracy_score',
                                    'average_odds_difference',
                                    'statistical_parity_difference',
                                    'equal_opportunity_difference',
                                    'disparate_impact'
                                   ]
          }

hyperparameters = {'LogisticRegression':{'penalty': ['l1', 'l2'], 'C': [0.1, 0.5, 1, 1.5]},
                   'RandomForestClassifier':{'n_estimators': ['warn', 10, 20, 30, 40, 50, 100]
                  }

thresholds = [i * 10.0/100 for i in range(5)]

processor_args = {'unprivileged_groups': unprivileged, 'privileged_groups': privileged}

preprocessors=[DisparateImpactRemover()]
postprocessors=[CalibratedEqOddsPostprocessing(**processor_args)]



# Step 5: Run the grid search

Now that we've set all the parameters necessary for the grid search, we're ready to run it. The output of the grid search is saved to a .csv file.

Below we provide code that creates and uses the `ModelSearch` object to run a grid search over the parameters we specified and saves the output to a .csv file in the specified directory. **The search take a while to complete. Wait until the search completes before attempting to execute more cells.**

**Note: warnings may appear during search, however, as long as you don’t see any code errors it is fine to continue.**

In [5]:
Search = ModelSearch(models, metrics, hyperparameters, thresholds)
Search.grid_search(dataset, privileged=privileged, unprivileged=unprivileged, preprocessors=preprocessors, postprocessors=postprocessors)

Search.to_csv("search_output.csv")

/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bjohnson/anaconda3/envs/fairkit-learn/lib/python3

# Step 6: Render visualization of search results

Along with the ability to run a grid search, fairkit-learn also provides functionality to visualize the results of the grid search. Fairkit-learn uses Bokeh to render a visualization within the notebook, which you can use when completing the next task to explore trained models' performance and fairness.

The visualzation includes a graph that plots the search results that are pareto optimal. Each data point in the graph is a model with its own settings (e.g., hyper-parameters, pre/post processing). Each model class has its own color to make it easier to see which models are being shown in the visualization. To get more information on each model's settings, hover over the data point of interest; a tooltip will pop up with model settings.

Within the visualization, you can control what metrics and models are being included in the visualization. The drop down menus allow you to specify the x and y axes for the graph. The checklist below the list of models allows you to select which metrics can be considered in the graph. 

To view the *Pareto frontier* for any two metrics (e.g., accuracy and disparate impact), select those two metrics from the drop down menu and **only** check those boxes in the checklist

Below we provide code to load the search results file into Colabs, load Bokeh,  and plot the results from the search in the interactive plot.

In [6]:
# Import packages for visualization
from bokeh.io import output_notebook
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

# load Bokeh
output_notebook()

Loading BokehJS ...

In [11]:
from fklearn.interface.plot import *
import os, fklearn

# Define function that takes in a document and attaches the bokeh server to it
def modify_doc(doc):
    
    # Load custom styles (for notebook only)
    style = os.path.join(fklearn.__path__[0], 'interface', 'static', 'css', 'styles-notebook.css')
    custom_css = Div(text="<link rel='stylesheet' type='text/css' href=" + style + ">")
    add_btn = Button(label="Add Plot", button_type="success")
    remove_btn = Button(label="Remove Plot", button_type="danger")

    explanations = os.path.join(fklearn.__path__[0], 'interface', 'static', 'data', 'explanations.json')
    
    # Construct our viewport
    l = layout([
        [custom_css],
        create_plot("search_output.csv", explanations)
    ], sizing_mode="fixed", css_classes=["layout-container"])

    doc.add_root(l)
    
# Set up the application
handler = FunctionHandler(modify_doc)
app = Application(handler)

# Render visualization in the notebook
show(app)